# ```tf.estimator.Estimator``` class
Estimators encapsulate the following actions:

* training
* evaluation
* prediction
* export for serving

## Benefits of Estimators
* Estimators are abstracted out from single or multi-node setup on CPUs, GPUs, TPUs (and FPGAs in the future)
* Simplify sharing your work
* Estimators are build on top of ```ft.layer```, thus customization is simple
* You don't have to build the graph, it is build for you

But you need:
* Separate data input pipline and model

Pre-made Estimators create and manage ```Graph``` and ```Session``` objects for you.

## Structure of a pre-made Estimators program
1. Write one or more dataset importing functions
2. Define the feature columns
3. Instantiate the relevant pre-made Estimator
4. Call a training, evaluation, or inference method


## 1. Write one or more dataset importing functions
    
Each dataset importing function must return two objects:
* a dictionary in which the keys are feature names and the values are feature Tensors
* labels tensor


In [2]:
def input_fn(dataset):
    feature_dict = {}
    label = []
    # ...  manipulate dataset, extracting feature names and the label
    return feature_dict, label

In [ ]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
  features = data["features"]
  labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

* embed the ```features``` and ```labels``` arrays in the TensorFlow graph as ```tf.constant()``` operations
* works well for a small datase
* data from ```features``` and ```labels``` replicated twice in memory
* Careful!!! Can run into the 2GB limit for the ```tf.GraphDef``` protocol buffer

Alternatively,
* define the ```Dataset``` in terms of ```tf.placeholder()``` tensors
* feed the NumPy arrays during initialization of ```Iterator``` over the dataset.

In [ ]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
  features = data["features"]
  labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
# [Other transformations on `dataset`...]
dataset = ...
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})

### Decoding image data and resizing it

In [ ]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_image(image_string)
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label

# A vector of filenames.
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])

# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant([0, 37, ...])

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)

### Simple batching

In [ ]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
print(sess.run(next_element))  # ==> ([4, 5, 6,   7],   [-4, -5,  -6,  -7])
print(sess.run(next_element))  # ==> ([8, 9, 10, 11],   [-8, -9, -10, -11])

## 2. Define the feature columns
Each ```tf.feature_column``` identifies a feature name, its type, and any input pre-processing.

In [ ]:
# Define three numeric feature columns.
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column('median_education',
                    normalizer_fn='lambda x: x - global_education_mean')

## 3. Instantiate the relevant pre-made Estimator

In [ ]:
# Instantiate an estimator, passing the feature columns.
estimator = tf.estimator.Estimator.LinearClassifier(
    feature_columns=[population, crime_rate, median_education],
    )

## 4. Call a training, evaluation, or inference method

In [ ]:
# my_training_set is the function created in Step 1
estimator.train(input_fn=my_training_set, steps=2000)

In [ ]:
# Instantiate a Keras inception v3 model.
keras_inception_v3 = tf.keras.applications.inception_v3.InceptionV3(weights=None)
# Compile model with the optimizer, loss, and metrics you'd like to train with.
keras_inception_v3.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),
                          loss='categorical_crossentropy',
                          metric='accuracy')
# Create an Estimator from the compiled Keras model. Note the initial model
# state of the keras model is preserved in the created Estimator.
est_inception_v3 = tf.keras.estimator.model_to_estimator(keras_model=keras_inception_v3)

# Treat the derived Estimator as you would with any other Estimator.
# First, recover the input name(s) of Keras model, so we can use them as the
# feature column name(s) of the Estimator input function:
keras_inception_v3.input_names  # print out: ['input_1']
# Once we have the input name(s), we can create the input function, for example,
# for input(s) in the format of numpy ndarray:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"input_1": train_data},
    y=train_labels,
    num_epochs=1,
    shuffle=False)
# To train, we call Estimator's train function:
est_inception_v3.train(input_fn=train_input_fn, steps=2000)